In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

In [4]:
import base64
import os
import mimetypes
from google import genai
from google.genai import types


def save_binary_file(file_name, data):
    f = open(file_name, "wb")
    f.write(data)
    f.close()


def generate():
    client = genai.Client(
        api_key=os.getenv("GEMINI_API_KEY"),
    )

    model = "gemini-2.0-flash-exp-image-generation"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text="""
You are an expert in generating **realistic** vertical images (1080x1920) for YouTube Shorts. Your task is to create visually accurate, natural-looking scenes that follow real-world lighting, textures, and proportions. Focus on realism in facial features, backgrounds, materials, and colors. The final image should feel like a high-resolution photograph taken with a professional camera.
"""
                ),
            ],
        ),
        types.Content(
            role="model",
            parts=[
                types.Part.from_text(
                    text="""
Understood. Please provide the scene description, characters, mood, and other elements you want visualized.
"""
                ),
            ],
        ),
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text="""
Scene: A charming, sun-dappled scene in a park or grassy area. A large, leafy tree is the central element, providing shade.
Character: A young boy, around 8–10 years old, with slightly tousled brown hair and bright, intelligent eyes. He's wearing a t-shirt and shorts. He's sitting cross-legged or leaning on the tree trunk.
Action: He is reading a book with a subtle smile. His posture shows confidence and peace.
Mood: Serene and joyful. The scene should radiate warmth and love for reading.
Animation Hints: Subtle tree rustling, slight hair movement, occasional page turn, soft eye motion.
Colors: Warm greens, browns, and golden sunlight. Natural and calm.
Framing: Vertical 1080x1920 format, optimized for YouTube Shorts. Center the boy and tree with gentle background detail.
"""
                ),
            ],
        ),
    ]

    generate_content_config = types.GenerateContentConfig(
        response_modalities=[
            "image",
            "text",
        ],
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        if (
            not chunk.candidates
            or not chunk.candidates[0].content
            or not chunk.candidates[0].content.parts
        ):
            continue
        if chunk.candidates[0].content.parts[0].inline_data:
            file_name = "scene_image"
            inline_data = chunk.candidates[0].content.parts[0].inline_data
            file_extension = mimetypes.guess_extension(inline_data.mime_type)
            save_binary_file(f"{file_name}{file_extension}", inline_data.data)
            print(
                "File of mime type" f" {inline_data.mime_type} saved" f"to: {file_name}"
            )
        else:
            print(chunk.text)


if __name__ == "__main__":
    generate()

Generating
 a realistic vertical image (1080x1920) for
 a YouTube Short depicting a serene park scene. The central focus is a large,
 leafy tree casting dappled sunlight onto the ground. An 8–10-year-old boy with slightly tousled brown hair and bright eyes is sitting
 either cross-legged or leaning against the tree trunk, engrossed in a book. He wears a simple t-shirt and shorts and has a subtle, confident
 smile. The overall mood is joyful and peaceful, with warm greens and browns dominating the color palette, accented by golden sunbeams filtering through the leaves. The framing is a vertical composition suitable for YouTube Shorts, centering the boy and tree with a
 softly blurred background.


File of mime type image/png savedto: scene_image
